In [261]:
import pandas as pd
import numpy as np

1. merging 10 US vital data sets and cleaning

In [262]:
df1 = pd.read_csv("Underlying Cause of Death, 2006.txt",  delimiter="\t")
df2 = pd.read_csv("Underlying Cause of Death, 2007.txt",  delimiter="\t")
df3 = pd.read_csv("Underlying Cause of Death, 2008.txt",  delimiter="\t")
df4 = pd.read_csv("Underlying Cause of Death, 2009.txt",  delimiter="\t")
df5 = pd.read_csv("Underlying Cause of Death, 2010.txt",  delimiter="\t")
df6 = pd.read_csv("Underlying Cause of Death, 2011.txt",  delimiter="\t")
df7 = pd.read_csv("Underlying Cause of Death, 2012.txt",  delimiter="\t")
df8 = pd.read_csv("Underlying Cause of Death, 2013.txt",  delimiter="\t")
df9 = pd.read_csv("Underlying Cause of Death, 2014.txt",  delimiter="\t")
df10 = pd.read_csv("Underlying Cause of Death, 2015.txt",  delimiter="\t")

In [263]:
# merged 10 data sets

df=[df1,df2,df3, df4, df5, df6, df7, df8, df9, df10]
df_new=pd.concat(df)

In [264]:
df_new.head()

,Notes,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths
0,NaN,"Autauga County, AL",1001.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,449
1,NaN,"Baldwin County, AL",1003.0,2006.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,D1,11
2,NaN,"Baldwin County, AL",1003.0,2006.0,2006.0,All other alcohol-induced causes,A9,15
3,NaN,"Baldwin County, AL",1003.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,1613
4,NaN,"Barbour County, AL",1005.0,2006.0,2006.0,All other non-drug and non-alcohol causes,O9,284


In [265]:
#split state and county name

temp=df_new['County'].str.split(",",expand=True)
df_new['County']=temp[0]
df_new['State']=temp[1]

In [266]:
df_new['Drug/Alcohol Induced Cause'].unique()

array(['All other non-drug and non-alcohol causes',
       'Drug poisonings (overdose) Unintentional (X40-X44)',
       'All other alcohol-induced causes',
       'All other drug-induced causes',
       'Drug poisonings (overdose) Undetermined (Y10-Y14)',
       'Drug poisonings (overdose) Suicide (X60-X64)', nan,
       'Alcohol poisonings (overdose) (X45, X65, Y15)',
       'Drug poisonings (overdose) Homicide (X85)'], dtype=object)

In [267]:
#select 'Drug poisonings (overdose) Unintentional (X40-X44)', 'Drug poisonings (overdose) Undetermined (Y10-Y14)', 'Drug poisonings (overdose) Suicide (X60-X64)'
df_death=df_new.loc[df_new['Drug/Alcohol Induced Cause'].isin(['Drug poisonings (overdose) Unintentional (X40-X44)', 'Drug poisonings (overdose) Undetermined (Y10-Y14)', 'Drug poisonings (overdose) Suicide (X60-X64)'])]

In [268]:
#reorder col names
columnsTitles = ['Notes', 'State', 'County', 'County Code', 'Year', 'Year Code',
       'Drug/Alcohol Induced Cause', 'Drug/Alcohol Induced Cause Code',
       'Deaths']

df_death = df_death.reindex(columns=columnsTitles).copy()

In [269]:
#drop useless cols

df_death=df_death.drop(["Notes", "Year Code", "Drug/Alcohol Induced Cause Code"],axis=1 )

In [270]:
df_death.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097.0,2006.0,Drug poisonings (overdose) Undetermined (Y10-Y14),10


In [271]:
#cleaning missing value

df_death['Deaths']=df_death['Deaths'].astype(str)
df_death= df_death[~df_death['Deaths'].str.contains("Missing")]



In [272]:
#change data type

df_death['County Code']=df_death['County Code'].astype(int)
df_death['Year']=df_death['Year'].astype(int)


In [273]:
df_death['Deaths']=pd.to_numeric(df_death['Deaths'], downcast='integer')

In [274]:
df_death.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths
1,AL,Baldwin County,1003,2006,Drug poisonings (overdose) Unintentional (X40-...,11
12,AL,Chilton County,1021,2006,Drug poisonings (overdose) Unintentional (X40-...,13
39,AL,Jefferson County,1073,2006,Drug poisonings (overdose) Unintentional (X40-...,55
55,AL,Mobile County,1097,2006,Drug poisonings (overdose) Unintentional (X40-...,23
56,AL,Mobile County,1097,2006,Drug poisonings (overdose) Undetermined (Y10-Y14),10


2. population data cleaning

In [275]:
df2= pd.read_csv("pop_counties_20062012.csv")

In [276]:
df2.head()

,BUYER_COUNTY,BUYER_STATE,countyfips,STATE,COUNTY,county_name,NAME,variable,year,population
0,AUTAUGA,AL,1001,1,1,Autauga,"Autauga County, Alabama",POPESTIMATE2006,2006,51328.0
1,BALDWIN,AL,1003,1,3,Baldwin,"Baldwin County, Alabama",POPESTIMATE2006,2006,168121.0
2,BARBOUR,AL,1005,1,5,Barbour,"Barbour County, Alabama",POPESTIMATE2006,2006,27861.0
3,BIBB,AL,1007,1,7,Bibb,"Bibb County, Alabama",POPESTIMATE2006,2006,22099.0
4,BLOUNT,AL,1009,1,9,Blount,"Blount County, Alabama",POPESTIMATE2006,2006,55485.0


In [277]:
#split state and county name, and cleaning

temp=df2['NAME'].str.split(",",expand=True)
df2['County']=temp[0]
df2['State']=temp[1]

In [278]:
df2 =df2.drop(["BUYER_COUNTY", "STATE", "COUNTY", "NAME", "variable", 'State', 'county_name'],axis=1 )

In [279]:
df2.rename(columns = {'year':'Year', 'countyfips':'County Code', 'BUYER_STATE':'State' }, inplace = True)

In [280]:
df2=df2.dropna(axis=0, how='any')

3. Final data set

In [281]:
#merging df_death and df2 to get final data set

df_final=pd.merge(df_death, df2, how='inner', on=['County Code', 'Year'])

In [282]:
df_final['population']=df_final['population'].astype(int)

In [283]:
df_final=df_final.drop(["State_y", "County_y"],axis=1 )

In [284]:
df_final.rename(columns = {'State_x':'State', 'County_x':'County'}, inplace = True)

In [285]:
df_final.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths,population
0,AL,Baldwin County,1003,2006,Drug poisonings (overdose) Unintentional (X40-...,11,168121
1,AL,Chilton County,1021,2006,Drug poisonings (overdose) Unintentional (X40-...,13,42318
2,AL,Jefferson County,1073,2006,Drug poisonings (overdose) Unintentional (X40-...,55,655893
3,AL,Mobile County,1097,2006,Drug poisonings (overdose) Unintentional (X40-...,23,402916
4,AL,Mobile County,1097,2006,Drug poisonings (overdose) Undetermined (Y10-Y14),10,402916


In [286]:
#Death rate Calculation

df_final['Death Rate']=df_final['Deaths']/df_final['population']

In [287]:
df_final.head()

,State,County,County Code,Year,Drug/Alcohol Induced Cause,Deaths,population,Death Rate
0,AL,Baldwin County,1003,2006,Drug poisonings (overdose) Unintentional (X40-...,11,168121,0.000065
1,AL,Chilton County,1021,2006,Drug poisonings (overdose) Unintentional (X40-...,13,42318,0.000307
2,AL,Jefferson County,1073,2006,Drug poisonings (overdose) Unintentional (X40-...,55,655893,0.000084
3,AL,Mobile County,1097,2006,Drug poisonings (overdose) Unintentional (X40-...,23,402916,0.000057
4,AL,Mobile County,1097,2006,Drug poisonings (overdose) Undetermined (Y10-Y14),10,402916,0.000025
